In [ ]:
import numpy as np
import pandas as pd
import re
fileName1 = "data1.npy" 
data1 = np.load(fileName1, allow_pickle=True)
#fileName2 = "data2.npy" 
#data2 = np.load(fileName2, allow_pickle=True)
#fileName3 = "data3.npy" 
#data3 = np.load(fileName3, allow_pickle=True)

In [9]:


def tidyGame(inputGame):
    fullGame=inputGame
    game = fullGame['liveData']['plays']['allPlays']
    #################################################(1/20)
    #Remove scheduled games that did not take place
    if len(game) == 0:
        return np.nan
    #################################################(2/20)
    #Populate array of eventType
    eventType=[]
    lastEventType=[]
    lastEventPeriod=[]
    lastEventPeriodTime=[]
    lastEventXCoord=[]
    lastEventYCoord=[]
    for x in game:  
        if str(x['result']['event']) == 'Shot' or str(x['result']['event']) == 'Goal':
            eventType.append(x['result']['event'])
            lastEventType.append(lastEvent['result']['event'])
            lastEventPeriod.append(lastEvent['about']['period'])
            lastEventPeriodTime.append(lastEvent['about']['periodTime'])
            if 'x' in lastEvent['coordinates']:
                lastEventXCoord.append(lastEvent['coordinates']['x'])
            else:
                lastEventXCoord.append(np.nan)
            if 'y' in lastEvent['coordinates']:
                lastEventYCoord.append(lastEvent['coordinates']['y'])
            else:
                lastEventYCoord.append(np.nan)
        lastEvent=x
    #################################################(3/20)
    #Populate array of Period
    period=[]
    for x in game:
        if str(x['result']['event']) == 'Shot' or str(x['result']['event']) == 'Goal':
            period.append(x['about']['period'])
    #################################################(4/20)
    #Populate array of periodTime
    periodTime=[]
    for x in game:  
        if str(x['result']['event']) == 'Shot' or str(x['result']['event']) == 'Goal':
            periodTime.append(x['about']['periodTime'])
    #################################################(5/20)
    #Populate array of periodType
    periodType=[]
    for x in game:  
        if str(x['result']['event']) == 'Shot' or str(x['result']['event']) == 'Goal':
            periodType.append(x['about']['periodType'])
    #################################################(6/20)
    #Populate array of gameID
    gameID=[fullGame['gamePk']]*(len(periodType))
    #################################################(7/20)
    #Populate array of teamOfShooter
    teamOfShooter=[]
    for x in game:  
        if str(x['result']['event']) == 'Shot' or str(x['result']['event']) == 'Goal':
            teamOfShooter.append(x['team']['name'])
    #################################################(8/20)
    #Populate array of homeOrAway
    homeOrAway=[]
    for x in game:  
        if str(x['result']['event']) == 'Shot' or str(x['result']['event']) == 'Goal':
            if str(x['team']['name'])==str(fullGame['gameData']['teams']['away']['name']):
                homeOrAway.append("away")
            if str(x['team']['name'])==str(fullGame['gameData']['teams']['home']['name']):
                homeOrAway.append("home")   
    #################################################(9/20)
    #Populate arrays of x and y coordinates
    xCoord=[]
    yCoord=[]
    for x in game:  
        if str(x['result']['event']) == 'Shot' or str(x['result']['event']) == 'Goal':       
            if 'x' in x['coordinates']:
                xCoord.append(x['coordinates']['x'])
            else:
                xCoord.append(np.nan)
            if 'y' in x['coordinates']:
                yCoord.append(x['coordinates']['y'])
            else:
                yCoord.append(np.nan)
    #################################################(10/20)
    #Populate array of shooter
    shooter=[]
    for x in game:  
        if str(x['result']['event']) == 'Shot' or str(x['result']['event']) == 'Goal':
            shooter.append(x['players'][0]['player']['fullName'])
    #################################################(11/20)
    #Populate array of Goalie
    goalie=[]
    for x in game:  
        if str(x['result']['event']) == 'Shot' or str(x['result']['event']) == 'Goal':
            goalie.append(x['players'][len(x['players'])-1]['player']['fullName'])
    #################################################(12/20)
    #Populate array of shotType
    shotType=[]
    for x in game:  
        if str(x['result']['event']) == 'Shot' or str(x['result']['event']) == 'Goal':
            if 'secondaryType' in x['result']:
                shotType.append(x['result']['secondaryType'])
            else:
                shotType.append(np.nan)
    #################################################(13/20)
    #Populate aray of emptyNet
    emptyNet=[]   
    for x in game:  
        if str(x['result']['event']) == 'Shot':
            emptyNet.append(np.nan)
        if str(x['result']['event']) == 'Goal':
            if 'emptyNet' in x['result']:
                emptyNet.append(x['result']['emptyNet'])
            else:
                emptyNet.append(np.nan)
    #################################################(14/20)
    #Populate array of strength
    strength=[]
    for x in game:  
        if str(x['result']['event']) == 'Shot':
            strength.append(np.nan)
        if str(x['result']['event']) == 'Goal':
            strength.append(x['result']['strength']['name'])
    #################################################(15/20)
    #Populate array of season
    season=[]
    for x in gameID:
        x=str(x)
        season.append(x[0:4])
    #################################################(16/20)
    #Populate array of rinkSide
    i=0
    rinkSide=[]
    for x in game:  
        if str(x['result']['event']) == 'Shot' or str(x['result']['event']) == 'Goal':
            if str(periodType[i])!="SHOOTOUT" and 'rinkSide' in fullGame['liveData']['linescore']['periods'][int(period[i])-1][str(homeOrAway[i])]:
                info=fullGame['liveData']['linescore']['periods'][int(period[i])-1][str(homeOrAway[i])]['rinkSide']
                rinkSide.append(info)
            else:
                rinkSide.append(np.nan)
            i+=1
    #################################################(17/20)
    #Populate array of gameType   
    gameType=[fullGame['gameData']['game']['type']]*(len(periodType)) 
    #################################################(18/20)
    #Populate array of totalPlayTime
    totalPlayTime=[]
    stringTotalPlayTime=""
    i=0 
    for y in fullGame['liveData']['plays']['allPlays']:
        i+=1
        if i==len(fullGame['liveData']['plays']['allPlays']):
        #IMPLEMENTING THE ACTUAL TOTAL PLAYTIME
            if fullGame['gameData']['game']['type'] == "R":
                if y['about']['period'] ==3:
                    stringTotalPlayTime="60:00"
                elif y['about']['period'] ==4:
                    extraTime=str(y['about']['periodTime'])
                    ex=extraTime.split(':')
                    minute=str(60+int(ex[0]))
                    stringTotalPlayTime= minute+":"+ex[1]
                elif y['about']['period'] ==5:
                    stringTotalPlayTime= "65:00"
            if fullGame['gameData']['game']['type'] == "P":
                if y['about']['period'] ==3:
                    stringTotalPlayTime="60:00"
                else:
                    prePeriod= int(y['about']['period'])-1
                    minute = str(prePeriod*2)+"0"
                    extraTime=str(y['about']['periodTime'])
                    ex=extraTime.split(':')
                    minute = str(int(minute)+int(ex[0]))
                    stringTotalPlayTime= minute+":"+ex[1]
                    
    dateformat=stringTotalPlayTime.split(':')

    if int(dateformat[0])>=60:
        if len(str(int(dateformat[0])-60))==2:

            if int(int(dateformat[0])/60) == 2:

                if int(dateformat[0]) >=120 and int(dateformat[0]) <= 129:         
                    stringTotalPlayTime = "0"+str(int(int(dateformat[0])/60))+":0"+str(int(dateformat[0])-120)+":"+str(dateformat[1])
                else:
                    stringTotalPlayTime = "0"+str(int(int(dateformat[0])/60))+":"+str(int(dateformat[0])-120)+":"+str(dateformat[1])
            else:

                stringTotalPlayTime = "0"+str(int(int(dateformat[0])/60))+":"+str(int(dateformat[0])-60)+":"+str(dateformat[1])


        if len(str(int(dateformat[0])-60))==1:
            stringTotalPlayTime = "0"+str(int(int(dateformat[0])/60))+":0"+str(int(dateformat[0])-60)+":"+str(dateformat[1]) 
    else:
        stringTotalPlayTime = "00:"+stringTotalPlayTime
    

    totalPlayTime = [stringTotalPlayTime]*(len(periodType))
    #################################################(19/20)
    #Transform lists into series
    eventType = pd.Series(eventType)
    period = pd.Series(period)
    periodTime = pd.Series(periodTime)
    periodType = pd.Series(periodType)
    gameID = pd.Series(gameID)
    teamOfShooter = pd.Series(teamOfShooter)
    homeOrAway = pd.Series(homeOrAway)
    xCoord = pd.Series(xCoord)
    yCoord = pd.Series(yCoord)
    shooter = pd.Series(shooter)
    goalie = pd.Series(goalie)
    shotType = pd.Series(shotType)
    emptyNet = pd.Series(emptyNet)
    strength = pd.Series(strength)
    season = pd.Series(season)
    gameType = pd.Series(gameType)
    totalPlayTime = pd.Series(totalPlayTime)
    lastEventType=pd.Series(lastEventType)
    lastEventPeriod=pd.Series(lastEventPeriod)
    lastEventPeriodTime=pd.Series(lastEventPeriodTime)
    lastEventXCoord=pd.Series(lastEventXCoord)
    lastEventYCoord=pd.Series(lastEventYCoord)
    #################################################(20/20)
    #Make dataframe with series
    df = pd.DataFrame({ 'eventType': eventType, 'period': period,'periodTime':periodTime,'periodType':periodType,'gameID':gameID,
                       'teamOfShooter':teamOfShooter,'homeOrAway':homeOrAway,'xCoord':xCoord,'yCoord':yCoord,'shooter':shooter,'goalie':goalie,
                       'shotType':shotType,'emptyNet':emptyNet,'strength':strength, 'season':season,'rinkSide':rinkSide,'gameType':gameType,'totalPlayTime':totalPlayTime,
                       'lastEventType':lastEventType,'lastEventPeriod':lastEventPeriod,'lastEventPeriodTime':lastEventPeriodTime,'lastEventXCoord':lastEventXCoord,'lastEventYCoord':lastEventYCoord})
    return df





def dfConcat(seasonList):
    df=seasonList[0]
    j=1
    for e in range(len(seasonList)-1):
        df=pd.concat([df,seasonList[j]], axis = 0)
        j+=1
    return df



In [10]:
'''
fileName1 = "data1.npy" 
data1 = np.load(fileName1, allow_pickle=True)
fileName2 = "data2.npy" 
data2 = np.load(fileName2, allow_pickle=True)
fileName3 = "data3.npy" 
data3 = np.load(fileName3, allow_pickle=True)
'''
bigList=[]

for x in data1:
    if type(tidyGame(x))!=type(np.nan):
        bigList.append(tidyGame(x))
for x in data2:
    if type(tidyGame(x))!=type(np.nan):
        bigList.append(tidyGame(x))
for x in data3:
    if type(tidyGame(x))!=type(np.nan):
        bigList.append(tidyGame(x))

In [11]:
s1=[]
s2=[]
s3=[]
s4=[]
s5=[]


i=0
for e in range(len(bigList)):
    if 'season' in bigList[i]:
        if int(bigList[i]['season'][0])==2016:
            s1.append(bigList[i])
        if int(bigList[i]['season'][0])==2017:
            s2.append(bigList[i])
        if int(bigList[i]['season'][0])==2018:
            s3.append(bigList[i])
        if int(bigList[i]['season'][0])==2019:
            s4.append(bigList[i])
        if int(bigList[i]['season'][0])==2020:
            s5.append(bigList[i])
        i+=1  

print(len(s1))
print(len(s2))
print(len(s3))
print(len(s4))
print(len(s5))

df1 = dfConcat(s1)
df2 = dfConcat(s2)
df3 = dfConcat(s3)
df4 = dfConcat(s4)
df5 = dfConcat(s5)

allSeasons = [df1,df2,df3,df4,df5]

dfTotal = dfConcat(allSeasons)

df1.to_csv("season2016.csv",index=False)
df2.to_csv("season2017.csv",index=False)
df3.to_csv("season2018.csv",index=False)
df4.to_csv("season2019.csv",index=False)
df5.to_csv("season2020.csv",index=False)
dfTotal.to_csv("Allseasons.csv",index=False)

1317
1355
1358
1168
952


In [8]:



stringTotalPlayTime= "150:27"
                    
dateformat=stringTotalPlayTime.split(':')

if int(dateformat[0])>=60:
    if len(str(int(dateformat[0])-60))==2:

        if int(int(dateformat[0])/60) == 2:

            if int(dateformat[0]) >=120 and int(dateformat[0]) <= 129:         
                stringTotalPlayTime = "0"+str(int(int(dateformat[0])/60))+":0"+str(int(dateformat[0])-120)+":"+str(dateformat[1])
            else:
                stringTotalPlayTime = "0"+str(int(int(dateformat[0])/60))+":"+str(int(dateformat[0])-120)+":"+str(dateformat[1])
        else:

            stringTotalPlayTime = "0"+str(int(int(dateformat[0])/60))+":"+str(int(dateformat[0])-60)+":"+str(dateformat[1])


    if len(str(int(dateformat[0])-60))==1:
        stringTotalPlayTime = "0"+str(int(int(dateformat[0])/60))+":0"+str(int(dateformat[0])-60)+":"+str(dateformat[1]) 
else:
    stringTotalPlayTime = "00:"+stringTotalPlayTime
        
print(stringTotalPlayTime)

02:30:27
